# Image Classification with CNN using Keras

## image classification

1.<a href = "#" >Import Libraries</a>

2.<a>Preprocess Data</a>

3.<a>Visualize Examples</a>

4.<a>Create Model</a>

5.<a>Train the Model</a>

6.<a>Final Predictions</a>

# Task 1: Import Libraries

In [ ]:
import tensorflow as tf
import os
import numpy as np
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense,Activation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from keras.utils import np_utils
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

# Task 2: Preprocess Data

In [ ]:
(x_train , y_train) ,(x_test , y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
print("x_train shape is : " , x_train.shape)
print("y_train shape is : " , y_train.shape)
print("x_test shape is : " , x_test.shape)
print("y_test shape is : " , y_test.shape)

In [ ]:
# x_train , y_train = get_three_classes(x_train , y_train)
# y_train[0]

# Task 3: Visualize Examples

In [ ]:
class_names = ['aeroplane', 'car', 'bird','cat','deer','dog','frog','horse','ship','truck']

## finction to return class name 

In [ ]:
# return class name
def class_name(label):
    return class_names[label]

In [ ]:
# visualize some of image
plt.figure(figsize = (20,10))
for i in range(10):
    plt.subplot(2,5,i+1)
    index = np.random.choice(x_train.shape[0] , 1 , replace =False)
    plt.imshow(x_train[int(index)])
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(class_name(int(y_train[index])))
    

In [ ]:
## convert y_train into one hot encoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(y_train)

In [ ]:
y_train.shape

In [ ]:
# convert y_test into one hot encoder

encoder = LabelEncoder()
encoder.fit(y_test)
y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(y_test)

In [ ]:
y_test[1000]

# Task 4: Create Model

In [ ]:
def create_model():
    def add_conv_block(model , num_filters):
        model.add(Conv2D(num_filters , 3 , activation = 'relu' , padding ='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(num_filters , 3 , activation = 'relu'))
        model.add(MaxPooling2D(pool_size = 2))
        model.add(Dropout(0.5))
        return model
    
    
    model = tf.keras.models.Sequential()
    model.add(Input(shape=(32,32,3)))
    model = add_conv_block(model ,32)
    model = add_conv_block(model ,64)
    model = add_conv_block(model ,128)
    model.add(Flatten())
    model.add(Dense(10 , activation = 'softmax'))
    
    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'adam' , metrics = ['acc']
        
    )
    
    return model

model = create_model()
model.summary()    

# Task 5: Train the Model

In [ ]:
h = model.fit(

    x_train/255.0 , y_train ,
    validation_data = (x_test/255.0 , y_test),
    epochs = 10 , batch_size = 128,
    callbacks = [
        EarlyStopping(monitor='val_accuracy'),
        ModelCheckpoint('model.h5', save_best_only=True ,monitor = 'val_accuracy')
        
        ]

)

# Task 6: Final Predictions

In [ ]:
test_pred = model.predict(x_test/255.0)
test_pred[0]

In [ ]:
y_test[5]

In [ ]:
np.argmax(y_test[5])

In [ ]:
## visualize some data

# visualize some of image
plt.figure(figsize = (20,10))
for i in range(10):
    plt.subplot(2,5,i+1)
    index = np.random.choice(x_test.shape[0] , 1 , replace =False)
    plt.imshow(x_test[int(index)])
    plt.xticks([])
    plt.yticks([])

    if np.argmax(test_pred[index]) == np.argmax(y_test[index]):

      plt.xlabel(class_name(np.argmax(test_pred[index]))  , color = 'green')
    else:
      plt.xlabel(class_name(np.argmax(test_pred[index]))  , color = 'red')



In [ ]:
np.argmax(test_pred[0])

In [ ]:
accs = h.history['acc']
val_acc = h.history['val_acc']

plt.plot(range(len(accs)) , accs , label = 'trainning')
plt.plot(range(len(accs)) , val_acc , label = 'validation')
plt.legend()

In [ ]:
train_pred = model.predict(x_train)

In [ ]:
# draw confusion matrix

matrix = metrics.confusion_matrix(y_test.argmax(axis=1), test_pred.argmax(axis=1))
matrix